# Colorado cannabis data

In this notebook, we're going to look at some county-level data on retail and medical cannabis sales in Colorado. [The data are](https://www.colorado.gov/pacific/revenue/colorado-marijuana-sales-reports) updated every month by the Colorado Department of Revenue.

The cannabis sales data lives here: `../data/co-cannabis-sales.csv`.

A few things to note about this data:

- Every row in our data is the sum of one month of sales for one category of cannabis ("retail" or "medical") for one county
- Not every county in Colorado has pot shops
- Not every county in Colorado has _retail_ pot shops
- To maintain taxpayer privacy, the state releases aggregate sales data only for counties with at least three dispensaries, and then only if none represent more than 80 percent of total sales, according to the Colorado Department of Revenue. Totals for counties that don't meet these criteria are represented in the data as 'NR'
- One of the "counties" in the data is "Sum of NR Counties" -- the weed sales from all of the "NR" counties grouped together for that month -- which is how everything totals like it should

We also have a CSV of Colorado county population estimates for 2016: `../data/co-county-pop.csv`. We'll use this to help us answer a question about per-capita sales.

Let's load up our data.

First, we'll import pandas. Then we'll tell pandas to change the way it displays floating-point numbers (decimals, basically) so that we won't have to look at big numbers in scientific notation later on (gross, no thanks).

Then we'll use the `read_csv()` method to create dataframes for each CSV as we need them.

In [6]:
import pandas as pd

In [21]:
# display floats with thousand-separator commas and no decimal points
pd.options.display.float_format = '{:,.2f}'.format

In [22]:
# dataframe from the sales CSV
df_sales = pd.read_csv('../data/co-cannabis-sales.csv')

In [23]:
# use `head()` to check the output
df_sales.head()

,month,year,county,amount,sales_type
0,4,2016,Adams,"387,146.00",medical
1,4,2016,Adams,"3,187,313.00",retail
2,4,2016,Alamosa,nan,medical
3,4,2016,Arapahoe,"7,531,661.00",retail
4,4,2016,Arapahoe,"1,682,301.00",medical


### Noodling

Let's check out the unique values in the columns, run some summary stats, check out samples, etc.

In [10]:
# check month values
# https://docs.python.org/3/howto/sorting.html#sorting-basics
sorted(df_sales.month.unique())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [11]:
# check year values
sorted(df_sales.year.unique())

[2014, 2015, 2016, 2017, 2018]

In [12]:
# check county values
sorted(df_sales.county.unique())

['Adams',
 'Alamosa',
 'Arapahoe',
 'Archuleta',
 'Boulder',
 'Chaffee',
 'Clear Creek',
 'Conejos',
 'Costilla',
 'Denver',
 'Eagle',
 'El Paso',
 'Fremont',
 'Garfield',
 'Gilpin',
 'Grand',
 'Gunnison',
 'Huerfano',
 'Jefferson',
 'La Plata',
 'Lake',
 'Larimer',
 'Las Animas',
 'Mesa',
 'Moffat',
 'Montezuma',
 'Montrose',
 'Morgan',
 'Otero',
 'Ouray',
 'Park',
 'Pitkin',
 'Pueblo',
 'Routt',
 'Saguache',
 'San Juan',
 'San Miguel',
 'Sedgwick',
 'Sum of NR Counties',
 'Summit',
 'Teller',
 'Weld']

In [13]:
# check type values
df_sales.sales_type.unique()

array(['medical', 'retail'], dtype=object)

In [25]:
# grab a sample
df_sales.sample()

,month,year,county,amount,sales_type
3082,3,2018,Adams,"455,735.00",medical


### Analysis

Let's answer some questions:

- Total sales for all years?
- Totals by year?
- Totals by county by year?
- Percent difference from 2014-2017, by county?
- Top counties in terms of per-capita retail sales for 2017? ([Checking this guy's work, basically](https://www.thecannabist.co/2018/02/09/colorado-marijuana-sales-southern-border/98669/).)

#### Total sales, all years

This one's pretty simple: Use `sum()`.

In [26]:
# sum the values in the `amount` column
df_sales.amount.sum()

4952920024.0

In [27]:
# peep a formatted version
'${:,}'.format(df_sales.amount.sum())

'$4,952,920,024.0'

#### Totals by year

For this, we'll select the two columns we're interested in ('year' and 'amount') and then use `groupby()` and `sum()`.

👉 For more details on grouping data in pandas, [check out this notebook](../reference/Grouping%20data%20in%20pandas.ipynb).

In [29]:
# select year, amount and groupby year, sum values
df_sales[['year', 'amount']].groupby('year').sum()

,amount
year,
2014,"683,523,739.00"
2015,"995,591,255.00"
2016,"1,307,203,473.00"
2017,"1,502,047,770.00"
2018,"464,553,787.00"


#### Totals by county and year

For this one, we'll need a pivot table. We're going to hand the [`pd.pivot_table()`](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.pivot_table.html) method five arguments:
- `df_sales`: The dataframe we're pivoting
- `index='county'`: The grouping column that will become the rows in our pivot table
- `values='amount'`: The column we're doing math on
- `aggfunc='sum'`: What aggregate function to apply to the values -- in this case, we want a sum
- `columns='year'`: The second grouping column that will become the columns in our pivot table

We'll fill null values with zeroes using the [`fillna()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) method. We're also going to save this one to a variable, `by_county_by_year`, because we'll use the pivot table to help us answer our next question.

In [121]:
by_county_by_year = pd.pivot_table(df_sales,
                                   index='county',
                                   values='amount',
                                   aggfunc='sum',
                                   columns='year').fillna(0)

by_county_by_year

year,2014,2015,2016,2017,2018
county,,,,,
Adams,"8,531,797.00","24,826,654.00","49,810,527.00","73,319,607.00","27,478,375.00"
Alamosa,0.00,0.00,0.00,0.00,0.00
Arapahoe,"19,083,212.00","68,533,185.00","113,890,135.00","124,313,953.00","40,809,345.00"
Archuleta,0.00,0.00,0.00,"6,397,396.00","2,480,149.00"
Boulder,"62,935,522.00","82,465,474.00","95,408,126.00","99,923,326.00","32,773,801.00"
Chaffee,"438,160.00","2,067,869.00","3,492,699.00","5,422,335.00","1,568,406.00"
Clear Creek,"6,095,765.00","6,435,080.00","7,414,079.00","7,262,477.00","2,476,363.00"
Conejos,0.00,0.00,"3,522,183.00","5,426,288.00","1,692,669.00"
Costilla,0.00,0.00,"2,836,421.00","3,874,954.00","1,128,973.00"


### Percent difference from 2014-2017, by county

So we're just going to build on the pivot table we just made by adding a calculated column.

First, though, we need to use [`reset_index()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html) to change the data structure from a pivot table with multiple levels to a plain old dataframe. Finally, we'll change the name of the indexed columns from `'year'` to `''`.

In [122]:
bcby_with_pct_change = by_county_by_year.reset_index()
bcby_with_pct_change.columns.name = ''

Now we can calculate the percent change over time. The formula is:

```
(new value - old value)
-----------------------    *   100
       old value
```

Then sort descending by our new value, and voilá!

In [124]:
bcby_with_pct_change['change14-17'] = ((bcby_with_pct_change[2017] - bcby_with_pct_change[2014]) / bcby_with_pct_change[2014]) * 100

bcby_with_pct_change.sort_values('change14-17', ascending=False)

,county,2014,2015,2016,2017,2018,change14-17
15,Grand,0.00,"1,380,291.00","3,292,625.00","3,957,091.00","1,261,007.00",inf
3,Archuleta,0.00,0.00,0.00,"6,397,396.00","2,480,149.00",inf
34,Saguache,0.00,0.00,0.00,"464,254.00","472,320.00",inf
22,Las Animas,0.00,"8,598,642.00","21,456,457.00","44,252,628.00","15,549,368.00",inf
7,Conejos,0.00,0.00,"3,522,183.00","5,426,288.00","1,692,669.00",inf
8,Costilla,0.00,0.00,"2,836,421.00","3,874,954.00","1,128,973.00",inf
30,Park,0.00,"1,815,652.00","3,331,577.00","4,250,202.00","1,351,164.00",inf
20,Lake,0.00,0.00,"1,559,663.00","2,521,582.00","859,746.00",inf
29,Ouray,0.00,"342,064.00","2,877,027.00","3,916,922.00","846,370.00",inf
27,Morgan,0.00,0.00,"4,138,315.00","7,918,703.00","2,633,517.00",inf


### Top counties, per-capita retail sales, 2017

To answer this one, we'll first need to join the cannabis sales data to the population data. Let's do that now.

First, read in the population CSV. We're going to specify that the FIPS code is a string, not a number, with the `dtype` argument.

In [126]:
# read in the pop data, specify fips column is a string
df_pop = pd.read_csv('../data/co-county-pop.csv', dtype={'fips': str})

In [127]:
# check the output with `head()`
df_pop.head()

,fips,county_name,pop_2016
0,08001,Adams,498187
1,08003,Alamosa,16654
2,08005,Arapahoe,637068
3,08007,Archuleta,12854
4,08009,Baca,3568


Now we need to isolate the sales data we're interested in: retail sales from 2017. I like to do this in two steps. First, filter to get retail sales. Then filter to get 2017 sales.

In [128]:
# filter for 'retail' sales
retail = df_sales[df_sales.sales_type == 'retail']

In [129]:
# check the output with `head()`
retail.head()

,month,year,county,amount,sales_type
1,4,2016,Adams,"3,187,313.00",retail
3,4,2016,Arapahoe,"7,531,661.00",retail
5,4,2016,Archuleta,nan,retail
7,4,2016,Boulder,"5,082,136.00",retail
9,4,2016,Chaffee,"300,638.00",retail


In [130]:
# filter that for 2017 sales
retail_2017 = retail[retail.year == 2017]

In [131]:
# check the output with `head()`
retail_2017.head()

,month,year,county,amount,sales_type
1436,7,2017,Adams,"5,462,382.00",retail
1438,7,2017,Arapahoe,"9,672,770.00",retail
1440,7,2017,Archuleta,"884,084.00",retail
1442,7,2017,Boulder,"7,070,534.00",retail
1444,7,2017,Chaffee,"662,413.00",retail


Excellent! Each row in this dataframe is a month's worth of sales, but we want to get the annual total. So we want to select the `county` and `amount` columns, group by `county` and `sum()` the amount:

In [132]:
retail_17_grouped = retail_2017[['county', 'amount']].groupby('county').sum()

In [133]:
# check the output with `head()`
retail_17_grouped.head()

,amount
county,
Adams,"62,132,216.00"
Arapahoe,"111,836,427.00"
Archuleta,"6,397,396.00"
Boulder,"77,438,466.00"
Chaffee,"5,422,335.00"


Perf. Now we'll join the two dataframes using the `merge()` function. (Note: Every county is represented in the population data, but not every county is present in the weed sales data.)

👉 For more details on merging data in pandas, [see this notebook](../reference/Merging%20data%20in%20pandas.ipynb).

In [134]:
merged = pd.merge(retail_17_grouped,
                  df_pop,
                  left_on='county',
                  right_on='county_name',
                  how='left')

In [135]:
# check the output with `head()`
merged.head()

,amount,fips,county_name,pop_2016
0,"62,132,216.00",08001,Adams,"498,187.00"
1,"111,836,427.00",08005,Arapahoe,"637,068.00"
2,"6,397,396.00",08007,Archuleta,"12,854.00"
3,"77,438,466.00",08013,Boulder,"322,226.00"
4,"5,422,335.00",08015,Chaffee,"19,058.00"


Now we can calculate the per-capita sales by dividing the amount into the population:

In [136]:
merged['per_capita'] = merged['amount'] / merged['pop_2016']

In [137]:
# `sort_values()` on per_capita column and check the output with `head()`
merged.sort_values('per_capita', ascending=False).head()

,amount,fips,county_name,pop_2016,per_capita
20,"43,913,648.00",08071,Las Animas,"14,103.00","3,113.78"
7,"3,874,954.00",08023,Costilla,"3,721.00","1,041.37"
24,"3,916,922.00",08091,Ouray,"4,857.00",806.45
34,"22,904,938.00",08117,Summit,"30,374.00",754.10
22,"20,163,593.00",08083,Montezuma,"26,999.00",746.83


Los Animas! Looks like the dude's numbers check out.